In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import os
import sys
import torch

sys.path.append('/hpc/home/ma618/cxrgen/')
sys.path.append('/hpc/home/ma618/cxrgen/src/')

In [2]:
from src.data import dataloaders

In [3]:
root = Path('/hpc/group/kamaleswaranlab/EmoryDataset/Images/chest_xrays')
dataset = root / 'longitudinal_data_corrected'
embedding_path = dataset / 'image_embeddings'
ehr_path = dataset / 'ehr_matrices'

In [4]:
supertable_path = root / 'matched_supertables_with_images'
sups = list(supertable_path.glob('*.pickle'))
len(sups)

244895

In [5]:
df = pd.read_pickle(sups[0])
df

,temperature,daily_weight_kg,height_cm,sbp_line,dbp_line,map_line,sbp_cuff,dbp_cuff,map_cuff,pulse,...,Albumin 5%_dose,infection,sepsis,CXR_ACC_NUM_1,vent_mode,vent_rate_set,vent_tidal_rate_set,vent_tidal_rate_exhaled,peep,vent_fio2
2016-05-30 07:29:53,36.6,60.0,NaN,NaN,NaN,NaN,72.000000,36.000000,NaN,75.000000,...,NaN,0,0,NaN,AC/CMV Volume,12,450,402,5,0.50
2016-05-30 08:29:53,36.6,54.1,188.0,NaN,NaN,NaN,134.000000,62.000000,NaN,53.000000,...,NaN,1,1,NaN,None,None,None,None,None,None
2016-05-30 09:29:53,36.0,NaN,NaN,NaN,NaN,NaN,107.333333,55.000000,74.666667,61.666667,...,NaN,1,1,NaN,AC/CMV Volume,15,450,453,5,0.4
2016-05-30 10:29:53,NaN,NaN,NaN,NaN,NaN,NaN,109.333333,57.333333,79.666667,59.000000,...,NaN,1,1,NaN,None,None,None,None,None,None
2016-05-30 11:29:53,NaN,NaN,NaN,NaN,NaN,NaN,114.000000,57.666667,79.666667,65.000000,...,NaN,1,1,NaN,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-14 09:29:53,37.7,NaN,NaN,NaN,NaN,NaN,122.000000,58.000000,83.000000,94.000000,...,NaN,1,1,00004DX160001032,None,None,None,None,None,None
2016-06-14 10:29:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.000000,...,NaN,1,1,00004DX160001032,None,None,None,None,None,None
2016-06-14 11:29:53,37.8,NaN,NaN,NaN,NaN,NaN,131.000000,66.000000,92.000000,92.000000,...,NaN,1,1,00004DX160001032,AC/CMV Volume,20,None,None,5,0.3
2016-06-14 12:29:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.500000,...,NaN,1,1,00004DX160001032,None,None,None,None,None,None


In [6]:
encounter_paths = list(ehr_path.glob("*.npy"))
prev_cxr_paths = list(embedding_path.glob("*_ffill_embeddings.npy"))
target_paths = list(embedding_path.glob("*_interpolated_embeddings.npy"))
len(encounter_paths), len(prev_cxr_paths), len(target_paths)

(17690, 17690, 17690)

In [7]:
encounter_paths.sort()
prev_cxr_paths.sort()
target_paths.sort()

In [8]:
batch_size = 8
model_type = 'transformer'
max_seq_length = 100
num_workers  = 8
shuffle = True


In [9]:
dataloader = dataloaders.create_encounter_dataloaders(encounter_paths=encounter_paths, 
                                        prev_cxr_paths=prev_cxr_paths,
                                        target_paths=target_paths,
                                        batch_size=batch_size,
                                        model_type=model_type,
                                        max_seq_length=max_seq_length,
                                        num_workers=num_workers,
                                        shuffle=shuffle)

In [10]:
batch = next(iter(dataloader))

In [11]:
batch['ehr'].shape, batch['prev_cxr'].shape, batch['target'].shape, batch['attention_mask'].shape

(torch.Size([8, 100, 81]),
 torch.Size([8, 100, 512]),
 torch.Size([8, 100, 512]),
 torch.Size([8, 100]))

In [12]:
import importlib
import src
import src.models.transformer as transformer
import src.models.transformernn as transformernn

from tqdm import tqdm
from src.training.trainer import TrainerConfig

In [13]:
import src.configs.config_def as cf
cf.set_paths()
configs = cf.run_configs()

In [14]:
importlib.reload(src.models.transformernn)

<module 'src.models.transformernn' from '/hpc/home/ma618/cxrgen/src/models/transformernn.py'>

In [24]:
config = {
        'ehr_dim': 81,
        'cxr_dim': 512,
        'd_model': 512,
        'num_encoder_layers': 6,
        'num_decoder_layers': 6,
        'num_heads': 8,
        'mlp_ratio': 4.0,
        'dropout': 0.1,
        'max_seq_length': 100
    }
    
# Create model
model = transformernn.create_transformer_model(config)

In [25]:
inf_mask = torch.isinf(batch['ehr'])
inf_indices = torch.where(inf_mask)
num_infs = inf_mask.sum().item()

if inf_mask.any():
    # For a multi-dimensional tensor
    for idx in zip(*torch.where(inf_mask)):
        print(f"Inf found at index: {idx}")

In [26]:
predictions = model(batch['ehr'],  batch['prev_cxr'], None, batch['attention_mask'], batch['attention_mask'], debug = True, batch_idx_debug = 4)

EHR input
tensor([[0.4167, 0.6767, 0.2889,  ..., 0.0000, 0.0000, 0.0000],
        [0.5000, 0.6767, 0.2667,  ..., 0.0000, 0.0000, 0.0000],
        [0.8000, 0.6767, 0.4667,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])
ehr - min: -6.5, max: 531.0, has_nan: False
EHR embedding
tensor([[ 1.1481e-01,  9.9440e-02, -2.7472e-02,  ...,  4.8887e-01,
          5.6493e-02, -2.2732e-01],
        [ 1.2922e-01,  3.0371e-02,  9.9204e-05,  ...,  4.7830e-01,
          7.8608e-02, -2.5675e-01],
        [ 1.1892e-01,  1.2091e-01,  5.4149e-02,  ...,  4.7943e-01,
          6.4979e-02, -3.3386e-01],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
      

/hpc/dctrl/ma618/torch/lib64/python3.9/site-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [27]:
predictions[4,:,:]

tensor([[ 0.0310,  0.4832, -0.2951,  ...,  1.2988, -0.2955, -0.4847],
        [ 0.2085,  0.5930, -0.3348,  ...,  0.7263, -0.0880, -0.3932],
        [ 0.1833,  0.3796, -0.3958,  ...,  0.9673, -0.2616, -0.2159],
        ...,
        [ 0.1569,  0.3412,  0.1895,  ...,  1.0137, -0.1192, -0.2396],
        [-0.0970,  0.6146,  0.1632,  ...,  0.7509, -0.0865, -0.1076],
        [ 0.0771,  0.6744, -0.0240,  ...,  0.6644, -0.0337, -0.1764]],
       grad_fn=<SliceBackward0>)

In [28]:
trainer_config = TrainerConfig(
        max_epochs=configs['num_epochs'],
        batch_size=configs['batch_size'],
        learning_rate=configs['lr'],
        weight_decay=0.01,
        lr_scheduler="cosine",
        warmup_ratio=0.1,
        checkpoint_dir= './',
        save_every=5,
        patience=15,
        grad_norm_clip=1.0,
        log_every=10,
        eval_every=1,
        mixed_precision=True,
        teacher_forcing_ratio=0.5,
        teacher_forcing_decay=0.98,  # Gradually reduce teacher forcing
    )

In [29]:
device = 'cpu'
        
# Create checkpoint directory

# Initialize teacher forcing ratio
teacher_forcing_ratio = trainer_config.teacher_forcing_ratio

# Setup optimizer
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=trainer_config.learning_rate,
    weight_decay=trainer_config.weight_decay,
)

# Setup learning rate scheduler
total_steps = len(dataloader) * trainer_config.max_epochs
warmup_steps = int(total_steps * trainer_config.warmup_ratio)

if trainer_config.lr_scheduler == "cosine":
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=total_steps - warmup_steps
    )
elif trainer_config.lr_scheduler == "linear":
    scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer, 
        start_factor=1.0,
        end_factor=0.1,
        total_iters=total_steps - warmup_steps
    )
elif trainer_config.lr_scheduler == "constant":
    scheduler = torch.optim.lr_scheduler.ConstantLR(
        optimizer, factor=1.0, total_iters=total_steps
    )
else:
    raise ValueError(f"Unknown scheduler: {trainer_config.lr_scheduler}")

# Warmup scheduler
warmup_scheduler = torch.optim.lr_scheduler.LinearLR(
    optimizer,
    start_factor=0.1,
    end_factor=1.0,
    total_iters=warmup_steps
)

# Setup loss function
criterion = torch.nn.MSELoss()

# Setup mixed precision training
scaler = torch.cuda.amp.GradScaler(enabled=trainer_config.mixed_precision)

# Setup model
model.to(device)

# Tracking variables
global_step = 0
best_val_loss = float('inf')
epochs_without_improvement = 0

/tmp/ipykernel_3200694/2604809632.py:49: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=trainer_config.mixed_precision)


In [32]:
model.train()
total_loss = 0.0
epoch = 1

with tqdm(dataloader, unit="batch", desc=f"Epoch {epoch}") as progress_bar:
        for batch_idx, batch in enumerate(progress_bar):
            # Move batch to device
            batch_encounters = batch['encounter_name']
            del batch['encounter_name']
            batch = {k: v.to(device) for k, v in batch.items()}
            
            batch['ehr'][torch.isinf(batch['ehr'])] = 0
            
            # Decide on teacher forcing
            use_teacher_forcing = torch.rand(1).item() < teacher_forcing_ratio
            target_input = batch["target"][:, :-1] if use_teacher_forcing else None

            # Forward pass with mixed precision
            with torch.cuda.amp.autocast(enabled=trainer_config.mixed_precision):
                outputs = model(
                    ehr=batch["ehr"],
                    prev_cxr=batch["prev_cxr"],
                    target_input=None,
                    encoder_attention_mask=batch.get("attention_mask"),
                    decoder_attention_mask=batch.get("attention_mask"),
                    causal_mask=True
                )
                loss = criterion(outputs, batch["target"])
            if torch.isnan(loss):
                break

            # Backward pass with gradient scaling
            optimizer.zero_grad()
            scaler.scale(loss).backward()

            # Gradient clipping
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), trainer_config.grad_norm_clip)

            # Update weights
            scaler.step(optimizer)
            scaler.update()

            # Update learning rate
            if global_step < warmup_steps:
                warmup_scheduler.step()
            else:
                scheduler.step()

            # Update metrics
            total_loss += loss.item()
            global_step += 1

            # Update progress bar
            progress_bar.set_postfix(loss=loss.item(), lr=optimizer.param_groups[0]["lr"])

            # Log metrics
            if batch_idx % trainer_config.log_every == 0:
                print({
                    "train/batch_loss": loss.item(),
                    "train/learning_rate": optimizer.param_groups[0]["lr"],
                    "train/global_step": global_step,
                })

avg_loss = total_loss / len(dataloader)

Epoch 1:   0%|          | 0/2212 [00:00<?, ?batch/s]/tmp/ipykernel_3200694/2749420202.py:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=trainer_config.mixed_precision):
Epoch 1:   0%|          | 1/2212 [00:04<2:30:02,  4.07s/batch, loss=5.97, lr=1.11e-5]

{'train/batch_loss': 5.967195510864258, 'train/learning_rate': 1.1106690777576851e-05, 'train/global_step': 136}


Epoch 1:   0%|          | 11/2212 [00:09<15:26,  2.38batch/s, loss=5.89, lr=1.12e-5] 

{'train/batch_loss': 5.893756866455078, 'train/learning_rate': 1.1188065099457504e-05, 'train/global_step': 146}


Epoch 1:   1%|          | 21/2212 [00:12<13:50,  2.64batch/s, loss=6.52, lr=1.13e-5]

...

Epoch 1: 100%|█████████▉| 2201/2212 [11:54<00:03,  2.97batch/s, loss=0.434, lr=2.9e-5] 

{'train/batch_loss': 0.4335782527923584, 'train/learning_rate': 2.9009041591319984e-05, 'train/global_step': 2336}


Epoch 1: 100%|█████████▉| 2211/2212 [11:57<00:00,  3.54batch/s, loss=0.91, lr=2.91e-5] 

{'train/batch_loss': 0.9099376797676086, 'train/learning_rate': 2.9090415913200643e-05, 'train/global_step': 2346}


Epoch 1: 100%|██████████| 2212/2212 [11:57<00:00,  3.08batch/s, loss=1.41, lr=2.91e-5]


In [33]:
batch['ehr'].shape, batch['prev_cxr'].shape

(torch.Size([2, 100, 81]), torch.Size([2, 100, 512]))

In [35]:
torch.cat([batch['ehr'],  batch['prev_cxr']], dim = -1).shape

torch.Size([2, 100, 593])

In [31]:
inf_mask = torch.isinf(batch['ehr'])
inf_indices = torch.where(inf_mask)
num_infs = inf_mask.sum().item()

if inf_mask.any():
    # For a multi-dimensional tensor
    for idx in zip(*torch.where(inf_mask)):
        print(f"Inf found at index: {idx}")

Inf found at index: (tensor(3), tensor(64), tensor(46))
Inf found at index: (tensor(3), tensor(65), tensor(46))
Inf found at index: (tensor(3), tensor(66), tensor(46))
Inf found at index: (tensor(3), tensor(67), tensor(46))
Inf found at index: (tensor(3), tensor(68), tensor(46))
Inf found at index: (tensor(3), tensor(69), tensor(46))
Inf found at index: (tensor(3), tensor(70), tensor(46))
Inf found at index: (tensor(3), tensor(71), tensor(46))
Inf found at index: (tensor(3), tensor(72), tensor(46))
Inf found at index: (tensor(3), tensor(73), tensor(46))
Inf found at index: (tensor(3), tensor(74), tensor(46))
Inf found at index: (tensor(3), tensor(75), tensor(46))
Inf found at index: (tensor(3), tensor(76), tensor(46))
Inf found at index: (tensor(3), tensor(77), tensor(46))
Inf found at index: (tensor(3), tensor(78), tensor(46))
Inf found at index: (tensor(3), tensor(79), tensor(46))
Inf found at index: (tensor(3), tensor(80), tensor(46))
Inf found at index: (tensor(3), tensor(81), tens

In [ ]:
batch['ehr'][1, :, 45]

In [36]:
df = pd.read_pickle(supertable_path / (batch_encounters[1] + '.pickle'))

In [43]:
df.values[:, 45]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, -0.5018181818181818, -0.5018181818181818, -0.505,
       -0.505, -0.505, -0.505, -0.505, -0.505, -0.505, -0.25, -0.25,
       -0.25, -0.25, -0.5, -0.5, -0.2515625, -0.2515625, -0.2515625,
       -0.2515625, -0.2550000000000001, -0.2550000000000001,
       -0.2550000000000001, -0.2575, -0.253125, -0.253125, -0.253125,
       -0.253125, -0.253125, -0.2625, -0.2625, -0.2625, -0.2625,
       -0.2666666666666667, -0.2666666666666667, -0.2666666666666667,
       -0.26875, -0.2583333333333334, -0.2583333333333334,
       -0.2583333333333334, -0.2583333333333334, -0.2583333333333334,
       -0.2625, -0.2625, -0.2625, -0.2666666666666667,
       -0.2666666666666667, -0.2666666666666667, -0.275, -0.26875,
       -0.26875, -0.26875, -0.26875, -0.2625, -0.2625, -0.2625, -0.2625,
       -0.25, -0.25, -0.2583333333333334, -0.25833333333

In [30]:
outputs = model(
                    ehr=batch["ehr"],
                    prev_cxr=batch["prev_cxr"],
                    target_input=None,
                    encoder_attention_mask=batch.get("attention_mask"),
                    decoder_attention_mask=batch.get("attention_mask"),
                    causal_mask=True, debug = True, batch_idx_debug = 1
                )

EHR input
tensor([[0.9000, 1.8933, 1.2533,  ..., 0.0000, 1.0000, 0.0000],
        [1.0100, 1.8933, 1.7400,  ..., 0.0000, 1.0000, 0.0000],
        [1.0500, 1.8933, 1.8000,  ..., 0.0000, 1.0000, 0.0000],
        ...,
        [0.5000, 1.7533, 1.4000,  ..., 0.0000, 1.0000, 0.0000],
        [0.5000, 1.7533, 1.4000,  ..., 0.0000, 1.0000, 0.0000],
        [0.5000, 1.7533, 1.6000,  ..., 0.0000, 1.0000, 0.0000]])
ehr - min: -3.4000000953674316, max: inf, has_nan: False
EHR embedding
tensor([[ 0.3397, -2.5103, -1.9462,  ..., -0.2305, -1.0521, -0.8516],
        [ 0.3448, -2.5688, -1.9900,  ..., -0.2090, -1.0538, -0.8895],
        [ 0.3552, -2.6524, -2.0044,  ..., -0.2060, -1.0753, -0.9462],
        ...,
        [-0.9441, -4.9256, -2.1909,  ...,  2.2478, -3.2096, -0.9323],
        [-0.9532, -4.9645, -2.2049,  ...,  2.2638, -3.2335, -0.9412],
        [-0.9052, -5.0178, -2.2416,  ...,  2.2919, -3.2731, -0.9600]],
       grad_fn=<SliceBackward0>)
x - min: -inf, max: inf, has_nan: False
CXR condition
